In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import stats
import matplotlib.pyplot as plt
import os
from scipy.stats import friedmanchisquare, wilcoxon

# Jonckheere–Terpstra might not exist on older SciPy; handle gracefully
try:
    from scipy.stats import jonckheere_terpstra
    HAS_JT = True
except Exception:
    HAS_JT = False

from scipy.stats import ConstantInputWarning
import warnings
warnings.filterwarnings("ignore", category=ConstantInputWarning)

In [2]:
def load_and_clean(path: Path):
    """Load sheets and forward-fill KG/Model labels."""
    screen = pd.read_excel(path, sheet_name="Screen")
    screen_c = pd.read_excel(path, sheet_name="Screen with C")
    h1 = pd.read_excel(path, sheet_name="H1-AM")
    h1_c = pd.read_excel(path, sheet_name="H1-AM&C")

    for df in (screen, screen_c, h1, h1_c):
        for col in ["KG", "Model"]:
            if col in df.columns:
                df[col] = df[col].ffill()

    return screen, screen_c, h1, h1_c

In [3]:
def prepare_screen_with_complexity(screen_c: pd.DataFrame) -> pd.DataFrame:
    """
    Add per-complexity SCREEN rates based on the known CQ split:
      - 12 Simple, 11 Moderate, 10 Complex (total 33).

    Full Validity already implies syntax recognized + satisfiable + deterministic.
    FV_rate is:
        Full Validity / (#CQs at that complexity)
    VS_rate is:
        Valid Syntax recognized / (#CQs at that complexity)
    """
    df = screen_c.copy()

    df["denom"] = df["Complexity"].map(CQS_PER_COMPLEXITY).astype(float)

    # Full-validity rate out of all CQs for that complexity
    df["FV_rate"] = df["Full Validity"] / df["denom"]
    # Syntax-recognized rate out of all CQs for that complexity
    df["VS_rate"] = df["Valid Syntax recognized"] / df["denom"]

    df["ComplexityLevel"] = df["Complexity"].map(
        {"Simple": 1, "Moderate": 2, "Complex": 3}
    )

    return df

In [4]:
def prepare_h1_with_complexity(h1_c: pd.DataFrame) -> pd.DataFrame:
    """
    Add numeric complexity level for H1 metrics.

    H1 rate metrics:
        syntax_ok_rate, satisfiable_rate, deterministic_rate, h1_overall.
    Other H1 metrics (semantic ones) are already over valid queries only.
    """
    df = h1_c.copy()
    df["ComplexityLevel"] = df["Complexity"].map(
        {"Simple": 1, "Moderate": 2, "Complex": 3}
    )
    return df

In [5]:
# =====================================================================
#  ANALYSIS FUNCTIONS
# =====================================================================

def analyze_screen(screen_c_clean: pd.DataFrame):
    print("\n=== Part A: SCREEN metrics (task adherence) ===\n")

    valid = screen_c_clean.dropna(subset=["FV_rate", "VS_rate", "Complexity"])

    # ----- Global Kruskal–Wallis tests on FV_rate and VS_rate -----
    fv_groups = [g["FV_rate"].values for _, g in valid.groupby("Complexity")]
    vs_groups = [g["VS_rate"].values for _, g in valid.groupby("Complexity")]

    if len(fv_groups) >= 2:
        kw_fv, kw_fv_p = stats.kruskal(*fv_groups)
        print("Global Kruskal–Wallis on FV_rate across Complexity:")
        print(f"  H = {kw_fv:.3f}, p = {kw_fv_p:.4f}\n")
    else:
        print("Not enough groups for Kruskal–Wallis on FV_rate.\n")

    if len(vs_groups) >= 2:
        kw_vs, kw_vs_p = stats.kruskal(*vs_groups)
        print("Global Kruskal–Wallis on VS_rate (syntax recognition) across Complexity:")
        print(f"  H = {kw_vs:.3f}, p = {kw_vs_p:.4f}\n")
    else:
        print("Not enough groups for Kruskal–Wallis on VS_rate.\n")

    # ----- Global Spearman correlations -----
    rho_fv, rho_fv_p = stats.spearmanr(
        valid["ComplexityLevel"], valid["FV_rate"]
    )
    print("Global Spearman between ComplexityLevel and FV_rate:")
    print(f"  rho = {rho_fv:.3f}, p = {rho_fv_p:.4f}\n")

    rho_vs, rho_vs_p = stats.spearmanr(
        valid["ComplexityLevel"], valid["VS_rate"]
    )
    print("Global Spearman between ComplexityLevel and VS_rate:")
    print(f"  rho = {rho_vs:.3f}, p = {rho_vs_p:.4f}\n")

    # ----- Per (KG, Model) Spearman correlations -----
    per_pair = []
    for (kg, model), grp in screen_c_clean.groupby(["KG", "Model"]):
        g = grp.dropna(subset=["FV_rate", "VS_rate", "ComplexityLevel"])
        if g["ComplexityLevel"].nunique() >= 2 and g["FV_rate"].notna().sum() >= 2:
            r_fv, p_fv = stats.spearmanr(g["ComplexityLevel"], g["FV_rate"])
            r_vs, p_vs = stats.spearmanr(g["ComplexityLevel"], g["VS_rate"])
            if not np.isnan(r_fv):
                per_pair.append(
                    {
                        "KG": kg,
                        "Model": model,
                        "rho_FV": r_fv,
                        "p_FV": p_fv,
                        "rho_VS": r_vs,
                        "p_VS": p_vs,
                    }
                )

    if per_pair:
        sc_df = pd.DataFrame(per_pair)
        print("Per (KG, Model) Spearman correlations for SCREEN (FV_rate & VS_rate):")
        print(sc_df.to_string(index=False))
        print()

        # Sign test: negative vs positive FV_rate correlations (ignoring zeros)
        neg = (sc_df["rho_FV"] < 0).sum()
        pos = (sc_df["rho_FV"] > 0).sum()
        n = neg + pos
        if n > 0:
            p_sign = stats.binomtest(neg, n, 0.5, alternative="greater").pvalue
            print("Sign test for FV_rate correlations (negative > positive):")
            print(f"  neg = {neg}, pos = {pos}, n = {n}, p = {p_sign:.4f}\n")
        else:
            print("No non-zero FV_rate correlations to test.\n")
    else:
        sc_df = pd.DataFrame()
        print("No usable per-(KG,Model) correlations for SCREEN.\n")

    return sc_df  # per-pair SCREEN correlations, for plotting/inspection

In [6]:
def analyze_h1(h1_c_clean: pd.DataFrame):
    print("\n=== Part B/C: H1 metrics (path reasoning) ===\n")

    metrics = [
        "syntax_ok_rate",
        "satisfiable_rate",
        "deterministic_rate",
        "h1_overall",
    ]

    # ----- Global Kruskal–Wallis per metric across complexity -----
    print("Global Kruskal–Wallis tests across Complexity for H1 metrics:")
    for metric in metrics:
        valid = h1_c_clean.dropna(subset=[metric])
        groups = [g[metric].values for _, g in valid.groupby("Complexity")]
        if len(groups) >= 2:
            stat, p = stats.kruskal(*groups)
            print(f"  {metric:22s} H = {stat:.3f}, p = {p:.4f}")
    print()

    # ----- Per (KG, Model) Spearman correlations vs Complexity -----
    per_pair = []
    for (kg, model), grp in h1_c_clean.groupby(["KG", "Model"]):
        if grp["ComplexityLevel"].nunique() < 2:
            continue
        for metric in metrics:
            g = grp.dropna(subset=[metric])
            if g.shape[0] >= 2:
                r, p = stats.spearmanr(g["ComplexityLevel"], g[metric])
                if not np.isnan(r):
                    per_pair.append(
                        {
                            "KG": kg,
                            "Model": model,
                            "metric": metric,
                            "rho": r,
                            "p": p,
                        }
                    )

    per_pair_df = pd.DataFrame(per_pair)
    if per_pair_df.empty:
        print("No usable per-(KG,Model) correlations for H1 metrics.\n")
        return per_pair_df

    print("Per (KG, Model) Spearman correlations for H1 metrics (first few rows):")
    print(per_pair_df.head(20).to_string(index=False))
    print()

    # ----- Summary of correlations by metric -----
    summary_corr = per_pair_df.groupby("metric")["rho"].agg(
        ["count", "mean", "median"]
    )
    print("Summary of Spearman rho per metric (H1):")
    print(summary_corr.to_string())
    print()

    # ----- Sign tests: negative vs positive correlations -----
    print("Sign tests for H1 metrics (is negative trend more common than positive?):")
    for metric, grp in per_pair_df.groupby("metric"):
        neg = (grp["rho"] < 0).sum()
        pos = (grp["rho"] > 0).sum()
        n = neg + pos
        if n > 0:
            p_val = stats.binomtest(neg, n, 0.5, alternative="greater").pvalue
            print(
                f"  {metric:22s} neg = {neg:2d}, pos = {pos:2d}, n = {n:2d}, p = {p_val:.4f}"
            )
        else:
            print(f"  {metric:22s} no non-zero correlations")
    print()

    # ----- KG-level comparison: Big vs Small -----
    print("Big vs Small KG comparison on H1 metrics (Mann–Whitney U):")
    small_big_results = []
    for metric in metrics:
        big_vals = h1_c_clean[h1_c_clean["KG"] == "Big"][metric].dropna()
        small_vals = h1_c_clean[h1_c_clean["KG"] == "Small"][metric].dropna()
        if len(big_vals) > 0 and len(small_vals) > 0:
            u_stat, p = stats.mannwhitneyu(big_vals, small_vals, alternative="two-sided")
            small_big_results.append(
                {
                    "metric": metric,
                    "Big_mean": big_vals.mean(),
                    "Small_mean": small_vals.mean(),
                    "U": u_stat,
                    "p": p,
                    "n_big": len(big_vals),
                    "n_small": len(small_vals),
                }
            )

    if small_big_results:
        kg_df = pd.DataFrame(small_big_results)
        print(kg_df.to_string(index=False))
        print()
    else:
        print("Not enough data for Big vs Small KG comparison.\n")

    return per_pair_df

In [7]:
# =====================================================================
#  PLOTTING FUNCTIONS
# =====================================================================

def plot_screen_boxplots(screen_c_clean: pd.DataFrame):
    """Boxplots of FV_rate and VS_rate by Complexity."""
    for metric in ["FV_rate", "VS_rate"]:
        plt.figure()
        data = [g[metric].values for _, g in screen_c_clean.groupby("Complexity")]
        labels = [name for name, _ in screen_c_clean.groupby("Complexity")]
        plt.boxplot(data)
        plt.xticks(range(1, len(labels) + 1), labels)
        plt.ylabel(metric)
        # plt.title(f"{metric} by Complexity")
        out_path = OUT_DIR / f"boxplot_SCREEN_{metric}.png"
        plt.savefig(out_path, bbox_inches="tight")
        plt.close()

In [8]:
def plot_screen_boxplots_by_kg(screen_c_clean: pd.DataFrame):
    """
    Boxplots of FV_rate and VS_rate by Complexity, separately for each KG.
    Saves one PNG per (metric, KG).
    """
    for metric in ["FV_rate", "VS_rate"]:
        for kg, df_kg in screen_c_clean.groupby("KG"):
            plt.figure()
            data = [g[metric].values for _, g in df_kg.groupby("Complexity")]
            labels = [name for name, _ in df_kg.groupby("Complexity")]
            plt.boxplot(data)
            plt.xticks(range(1, len(labels) + 1), labels)
            plt.ylabel(metric)
            # plt.title(f"{metric} by Complexity (KG={kg})")  # optional
            out_path = OUT_DIR / f"boxplot_SCREEN_{metric}_byKG_{kg}.png"
            plt.savefig(out_path, bbox_inches="tight")
            plt.close()

In [9]:
# =====================================================================
#  ADDITIONAL PLOTS: H1 metrics by KG × Complexity (ADDITIONS ONLY)
# =====================================================================

def plot_h1_boxplots_by_kg(h1_c_clean: pd.DataFrame):
    """
    For each KG, make boxplots of each H1 metric by Complexity.
    Saves one PNG per (metric, KG).
    """
    metrics = ["syntax_ok_rate", "satisfiable_rate", "deterministic_rate", "h1_overall"]

    for metric in metrics:
        for kg, df_kg in h1_c_clean.groupby("KG"):
            plt.figure()
            data = [g[metric].values for _, g in df_kg.groupby("Complexity")]
            labels = [name for name, _ in df_kg.groupby("Complexity")]
            plt.boxplot(data)
            plt.xticks(range(1, len(labels) + 1), labels)
            plt.ylabel(metric)
            plt.title(f"{metric} by Complexity (KG={kg})")
            out_path = OUT_DIR / f"boxplot_H1_{metric}_byKG_{kg}.png"
            plt.savefig(out_path, bbox_inches="tight")
            plt.close()


def plot_h1_boxplots_kg_effect_per_complexity(h1_c_clean: pd.DataFrame):
    """
    For each Complexity, make boxplots of each H1 metric by KG (Small vs Big).
    Saves one PNG per (metric, Complexity).
    """
    metrics = ["syntax_ok_rate", "satisfiable_rate", "deterministic_rate", "h1_overall"]
    kgs = ["Small", "Big"]

    for metric in metrics:
        for comp, df_comp in h1_c_clean.groupby("Complexity"):
            plt.figure()
            data = [df_comp[df_comp["KG"] == kg][metric].dropna().values for kg in kgs]
            plt.boxplot(data)
            plt.xticks([1, 2], kgs)
            plt.ylabel(metric)
            plt.title(f"{metric} by KG (Complexity={comp})")
            out_path = OUT_DIR / f"boxplot_H1_{metric}_KGeffect_{comp}.png"
            plt.savefig(out_path, bbox_inches="tight")
            plt.close()


# =====================================================================
#  ADDITIONAL STATS: H1 KG effect (Big vs Small) (ADDITIONS ONLY)
# =====================================================================

def _cliffs_delta(x: np.ndarray, y: np.ndarray) -> float:
    """
    Cliff's delta effect size for two independent samples.
    Returns delta in [-1, 1], where positive means x > y.
    """
    x = np.asarray(x)
    y = np.asarray(y)
    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]
    if len(x) == 0 or len(y) == 0:
        return np.nan

    # O(n*m) but small here
    gt = 0
    lt = 0
    for xi in x:
        gt += np.sum(xi > y)
        lt += np.sum(xi < y)
    return (gt - lt) / (len(x) * len(y))


def h1_kg_effect_stats(h1_c_clean: pd.DataFrame):
    """
    Summarize KG effect on H1 metrics:
      1) Overall Big vs Small (Mann–Whitney U) pooled across complexities
      2) Big vs Small within each Complexity
    Also computes Cliff's delta as a nonparametric effect size.
    Saves a CSV for inspection.
    """
    print("\n=== EXTRA: H1 KG effect summary (Big vs Small) ===\n")

    metrics = ["syntax_ok_rate", "satisfiable_rate", "deterministic_rate", "h1_overall"]
    complexities = ["Simple", "Moderate", "Complex"]

    rows = []

    # (A) Overall pooled across complexities
    for metric in metrics:
        big_vals = h1_c_clean[h1_c_clean["KG"] == "Big"][metric].dropna().values
        small_vals = h1_c_clean[h1_c_clean["KG"] == "Small"][metric].dropna().values
        if len(big_vals) > 0 and len(small_vals) > 0:
            u_stat, p = stats.mannwhitneyu(big_vals, small_vals, alternative="two-sided")
            # Cliff's delta: use (Big - Small) direction for interpretability
            delta = _cliffs_delta(big_vals, small_vals)
            rows.append(
                {
                    "scope": "overall",
                    "Complexity": "ALL",
                    "metric": metric,
                    "Big_mean": np.mean(big_vals),
                    "Small_mean": np.mean(small_vals),
                    "n_big": len(big_vals),
                    "n_small": len(small_vals),
                    "U": u_stat,
                    "p": p,
                    "cliffs_delta(Big-Small)": delta,
                }
            )

    # (B) Within each complexity
    for comp in complexities:
        df_comp = h1_c_clean[h1_c_clean["Complexity"] == comp]
        for metric in metrics:
            big_vals = df_comp[df_comp["KG"] == "Big"][metric].dropna().values
            small_vals = df_comp[df_comp["KG"] == "Small"][metric].dropna().values
            if len(big_vals) > 0 and len(small_vals) > 0:
                u_stat, p = stats.mannwhitneyu(big_vals, small_vals, alternative="two-sided")
                delta = _cliffs_delta(big_vals, small_vals)
                rows.append(
                    {
                        "scope": "within_complexity",
                        "Complexity": comp,
                        "metric": metric,
                        "Big_mean": np.mean(big_vals),
                        "Small_mean": np.mean(small_vals),
                        "n_big": len(big_vals),
                        "n_small": len(small_vals),
                        "U": u_stat,
                        "p": p,
                        "cliffs_delta(Big-Small)": delta,
                    }
                )

    if not rows:
        print("Not enough data to compute KG effect stats.\n")
        return None

    kg_effect_df = pd.DataFrame(rows)
    print(kg_effect_df.to_string(index=False))
    print()

    out_path = OUT_DIR / "kg_effect_H1_mannwhitney_cliffsdelta.csv"
    kg_effect_df.to_csv(out_path, index=False)
    print(f"Saved KG effect stats to: {out_path}\n")

    return kg_effect_df

In [10]:
def plot_h1_boxplots(h1_c_clean: pd.DataFrame):
    """Boxplots of each H1 metric by Complexity."""
    metrics = [
        "syntax_ok_rate",
        "satisfiable_rate",
        "deterministic_rate",
        "h1_overall",
    ]
    for metric in metrics:
        plt.figure()
        data = [g[metric].values for _, g in h1_c_clean.groupby("Complexity")]
        labels = [name for name, _ in h1_c_clean.groupby("Complexity")]
        plt.boxplot(data)
        plt.xticks(range(1, len(labels) + 1), labels)
        plt.ylabel(metric)
        plt.title(f"{metric} by Complexity")
        out_path = OUT_DIR / f"boxplot_H1_{metric}.png"
        plt.savefig(out_path, bbox_inches="tight")
        plt.close()

In [11]:
def plot_h1_trendlines(h1_c_clean: pd.DataFrame):
    """
    Line plots: ComplexityLevel vs metric for each Model (separate lines).
    One plot per metric. Big+Small KG together, but labeled by model.
    """
    metrics = [
        "syntax_ok_rate",
        "satisfiable_rate",
        "deterministic_rate",
        "h1_overall",
    ]
    for metric in metrics:
        plt.figure()
        for (kg, model), grp in h1_c_clean.groupby(["KG", "Model"]):
            # Sort by ComplexityLevel to make lines consistent
            g = grp.sort_values("ComplexityLevel")
            x = g["ComplexityLevel"].values
            y = g[metric].values
            if len(x) >= 2 and not np.all(np.isnan(y)):
                # Plot a line for this KG–model
                label = f"{kg}-{model}"
                plt.plot(x, y, marker="o", label=label)
        plt.xticks([1, 2, 3], ["Simple", "Moderate", "Complex"])
        plt.xlabel("Complexity")
        plt.ylabel(metric)
        plt.title(f"{metric} vs Complexity (per KG–Model)")
        # If too many models, legend can get big; you can comment this out if noisy
        plt.legend(fontsize="x-small", bbox_to_anchor=(1.05, 1), loc="upper left")
        out_path = OUT_DIR / f"trend_H1_{metric}.png"
        plt.savefig(out_path, bbox_inches="tight")
        plt.close()

In [12]:
def plot_h1_rho_heatmap(per_pair_df: pd.DataFrame):
    """
    Simple heatmap of rho values:
      rows: (KG,Model)
      cols: metrics
    using imshow.
    """
    if per_pair_df.empty:
        return

    metrics = [
        "syntax_ok_rate",
        "satisfiable_rate",
        "deterministic_rate",
        "h1_overall",
    ]
    # Pivot to (KG,Model) as index, metrics as columns
    pivot_data = []
    row_labels = []
    for (kg, model), grp in per_pair_df.groupby(["KG", "Model"]):
        row_labels.append(f"{kg}-{model}")
        row = []
        for m in metrics:
            sub = grp[grp["metric"] == m]
            if not sub.empty:
                row.append(sub["rho"].iloc[0])
            else:
                row.append(np.nan)
        pivot_data.append(row)

    data = np.array(pivot_data, dtype=float)

    plt.figure()
    # Mask NaNs by turning them into 0, but you can also leave them; they'll appear as default color
    nan_mask = np.isnan(data)
    data_display = np.where(nan_mask, 0.0, data)

    im = plt.imshow(data_display, aspect="auto")
    plt.colorbar(im, label="Spearman rho")
    plt.yticks(range(len(row_labels)), row_labels, fontsize="x-small")
    plt.xticks(range(len(metrics)), metrics, rotation=45, ha="right")
    plt.title("Spearman rho (Complexity vs H1 metrics)\nper KG–Model")
    out_path = OUT_DIR / "heatmap_H1_rho.png"
    plt.savefig(out_path, bbox_inches="tight")
    plt.close()

In [13]:
def summarize_model_sensitivity(per_pair_df: pd.DataFrame):
    """
    Summarize how sensitive each (KG,Model) is to complexity.
    Uses the Spearman rho values from H1 analysis.
    - Negative mean_rho  => performance tends to drop with complexity
    - Near-zero mean_rho => relatively robust
    """
    if per_pair_df.empty:
        print("No per-pair H1 correlations to summarize.\n")
        return None

    metrics = ["syntax_ok_rate", "satisfiable_rate", "deterministic_rate", "h1_overall"]

    # pivot to (KG,Model) x metric with rho values
    pivot = (
        per_pair_df
        .pivot_table(index=["KG", "Model"], columns="metric", values="rho")
        .reindex(columns=metrics)
    )

    # average rho across metrics as a simple sensitivity score
    pivot["mean_rho_across_H1"] = pivot.mean(axis=1)

    print("\n=== Model complexity sensitivity (mean Spearman rho across H1 metrics) ===")
    print("(More negative = more performance drop as complexity increases)\n")
    print(pivot.sort_values("mean_rho_across_H1").to_string())
    print()

    # save for later inspection
    out_path = OUT_DIR / "model_complexity_sensitivity_H1.csv"
    pivot.to_csv(out_path)
    print(f"Saved model sensitivity table to: {out_path}\n")

    return pivot

In [14]:
def effect_size_simple_vs_complex(h1_c_clean: pd.DataFrame):
    """
    Compute Simple vs Complex effect sizes (Cohen's d)
    for each H1 metric. This complements Kruskal–Wallis + rho
    by giving a magnitude of the drop.
    """
    print("\n=== Simple vs Complex effect sizes (H1 metrics) ===\n")

    metrics = ["syntax_ok_rate", "satisfiable_rate", "deterministic_rate", "h1_overall"]

    for metric in metrics:
        simple = h1_c_clean[h1_c_clean["Complexity"] == "Simple"][metric].dropna()
        complex_ = h1_c_clean[h1_c_clean["Complexity"] == "Complex"][metric].dropna()

        if len(simple) < 2 or len(complex_) < 2:
            print(f"{metric:22s}: not enough data for effect size.")
            continue

        mean_simple = simple.mean()
        mean_complex = complex_.mean()
        sd_simple = simple.std(ddof=1)
        sd_complex = complex_.std(ddof=1)

        # pooled standard deviation
        n1, n2 = len(simple), len(complex_)
        pooled_var = ((n1 - 1) * sd_simple**2 + (n2 - 1) * sd_complex**2) / (n1 + n2 - 2)
        pooled_sd = np.sqrt(pooled_var)

        d = (mean_simple - mean_complex) / pooled_sd if pooled_sd > 0 else np.nan

        print(
            f"{metric:22s}: "
            f"mean_Simple = {mean_simple:.3f}, mean_Complex = {mean_complex:.3f}, "
            f"Cohen_d = {d:.3f} (n1={n1}, n2={n2})"
        )
    print()

In [15]:
def correlate_screen_and_h1(sc_df: pd.DataFrame, per_pair_df: pd.DataFrame):
    """
    Connect SCREEN and H1:
    - Join per (KG,Model) FV_rate rho with H1 rhos.
    - Check whether models whose SCREEN performance drops with complexity
      also show strong H1 degradation.

    Uses:
      sc_df: output of analyze_screen (per-pair SCREEN correlations)
      per_pair_df: output of analyze_h1 (per-pair H1 correlations)
    """
    if sc_df.empty or per_pair_df.empty:
        print("Need non-empty SCREEN and H1 per-pair data for correlation.\n")
        return None

    # We mostly care about overall H1 behaviour; start with h1_overall
    h1_overall_rho = per_pair_df[per_pair_df["metric"] == "h1_overall"].copy()

    merged = h1_overall_rho.merge(
        sc_df[["KG", "Model", "rho_FV", "rho_VS"]],
        on=["KG", "Model"],
        how="inner",
    )

    if merged.empty:
        print("No overlapping (KG,Model) rows between SCREEN and H1.\n")
        return None

    print("\n=== Relationship between SCREEN and H1 complexity sensitivity ===\n")
    print("Merged (KG,Model) rows (first few):")
    print(merged.head().to_string(index=False))
    print()

    # Correlation of SCREEN FV_rate rho with H1 h1_overall rho
    r_fv, p_fv = stats.pearsonr(merged["rho_FV"], merged["rho"])
    r_vs, p_vs = stats.pearsonr(merged["rho_VS"], merged["rho"])

    print("Correlation between SCREEN sensitivity and H1 sensitivity (per KG–Model):")
    print(
        f"  rho_FV (SCREEN)  vs rho(h1_overall): r = {r_fv:.3f}, p = {p_fv:.4f}"
    )
    print(
        f"  rho_VS (SCREEN)  vs rho(h1_overall): r = {r_vs:.3f}, p = {p_vs:.4f}\n"
    )

    # Save merged table for inspection
    out_path = OUT_DIR / "screen_vs_h1_sensitivity.csv"
    merged.to_csv(out_path, index=False)
    print(f"Saved SCREEN–H1 sensitivity table to: {out_path}\n")

    return merged


In [16]:
def rank_models_by_complexity(h1_c_clean: pd.DataFrame):
    """
    For each complexity level, compute mean h1_overall per Model
    (aggregated over KGs) and print a ranking. This shows which models
    remain robust as graphs get more complex.
    """
    print("\n=== Model rankings by h1_overall for each Complexity ===\n")

    rows = []
    for comp, grp in h1_c_clean.groupby("Complexity"):
        tmp = (
            grp.groupby("Model")["h1_overall"]
            .mean()
            .reset_index()
            .rename(columns={"h1_overall": "mean_h1_overall"})
        )
        tmp["Complexity"] = comp
        rows.append(tmp)

    if not rows:
        print("No data to rank models.\n")
        return None

    ranking_df = pd.concat(rows, ignore_index=True)

    for comp in ["Simple", "Moderate", "Complex"]:
        sub = ranking_df[ranking_df["Complexity"] == comp].sort_values(
            "mean_h1_overall", ascending=False
        )
        print(f"--- {comp} ---")
        print(sub.to_string(index=False))
        print()

    out_path = OUT_DIR / "model_rankings_by_complexity_h1_overall.csv"
    ranking_df.to_csv(out_path, index=False)
    print(f"Saved rankings to: {out_path}\n")

    return ranking_df


In [17]:
# =====================================================================
#  EXTRA STATISTICAL ANALYSES (do not modify existing ones)
# =====================================================================

def friedman_test_h1(h1_c_clean: pd.DataFrame):
    """
    Friedman repeated-measures test across complexity levels, using
    (KG,Model) pairs as blocks.

    Tests whether Simple/Moderate/Complex differ for each H1 metric,
    treating each model as its own control.
    """
    print("\n=== EXTRA: Friedman Test (Repeated-Measures across models) ===\n")

    metrics = ["syntax_ok_rate", "satisfiable_rate", "deterministic_rate", "h1_overall"]

    for metric in metrics:
        # rows: (KG,Model)  cols: Simple/Moderate/Complex
        pivot = h1_c_clean.pivot_table(
            index=["KG", "Model"],
            columns="Complexity",
            values=metric
        ).dropna(subset=["Simple", "Moderate", "Complex"])

        if pivot.shape[0] < 3:
            print(f"{metric:22s}: not enough (KG,Model) with all three complexities.")
            continue

        S = pivot["Simple"].values
        M = pivot["Moderate"].values
        C = pivot["Complex"].values

        stat, p = friedmanchisquare(S, M, C)
        print(f"{metric:22s}: Friedman χ² = {stat:.3f}, p = {p:.4f}")
    print()


def jt_test_h1(h1_c_clean: pd.DataFrame):
    """
    Jonckheere–Terpstra trend test for ordered categories:
    tests monotonic *decreasing* trend Simple -> Moderate -> Complex
    for each H1 metric.

    This directly encodes H1's directional hypothesis.
    """
    print("\n=== EXTRA: Jonckheere–Terpstra Trend Test (monotonic decreasing) ===\n")

    if not HAS_JT:
        print("SciPy version does not have jonckheere_terpstra; skipping this test.\n")
        return

    metrics = ["syntax_ok_rate", "satisfiable_rate", "deterministic_rate", "h1_overall"]

    for metric in metrics:
        g_simple   = h1_c_clean[h1_c_clean["Complexity"] == "Simple"][metric].dropna()
        g_moderate = h1_c_clean[h1_c_clean["Complexity"] == "Moderate"][metric].dropna()
        g_complex  = h1_c_clean[h1_c_clean["Complexity"] == "Complex"][metric].dropna()

        groups = [g_simple, g_moderate, g_complex]

        if any(len(g) == 0 for g in groups):
            print(f"{metric:22s}: insufficient data for JT test.")
            continue

        jt_stat, p = jonckheere_terpstra(
            groups,
            alternative="decreasing"   # we expect performance to decrease as complexity increases
        )
        print(f"{metric:22s}: JT = {jt_stat:.3f}, p = {p:.4f}")
    print()


def kendalls_w_h1(h1_c_clean: pd.DataFrame):
    """
    Compute Kendall's W (coefficient of concordance) across complexity levels
    for each H1 metric.

    W in [0,1]; higher means stronger agreement among (KG,Model) pairs about
    the relative ranking of Simple/Moderate/Complex.
    """
    print("\n=== EXTRA: Kendall's W (agreement across complexity levels) ===\n")

    metrics = ["syntax_ok_rate", "satisfiable_rate", "deterministic_rate", "h1_overall"]

    for metric in metrics:
        pivot = h1_c_clean.pivot_table(
            index=["KG", "Model"],
            columns="Complexity",
            values=metric
        ).dropna(subset=["Simple", "Moderate", "Complex"])

        if pivot.shape[0] < 3:
            print(f"{metric:22s}: insufficient models for Kendall's W.")
            continue

        # ranks per row (each (KG,Model) ranks the three complexities)
        ranks = pivot.rank(axis=1, ascending=True).values  # shape: n × k
        n, k = ranks.shape

        # Sum of ranks for each column (complexity)
        R_j = np.sum(ranks, axis=0)
        R_bar = n * (k + 1) / 2.0

        S = np.sum((R_j - R_bar) ** 2)
        W = 12 * S / (n**2 * (k**2 - 1))

        print(f"{metric:22s}: Kendall's W = {W:.3f}")
    print()


def pairwise_wilcoxon_h1(h1_c_clean: pd.DataFrame):
    """
    Paired Wilcoxon tests per H1 metric:
      - Simple vs Moderate
      - Moderate vs Complex
      - Simple vs Complex

    Uses (KG,Model) as the paired unit.
    """
    print("\n=== EXTRA: Pairwise Wilcoxon tests (Simple vs Moderate vs Complex) ===\n")

    metrics = ["syntax_ok_rate", "satisfiable_rate", "deterministic_rate", "h1_overall"]

    for metric in metrics:
        pivot = h1_c_clean.pivot_table(
            index=["KG", "Model"],
            columns="Complexity",
            values=metric
        ).dropna(subset=["Simple", "Moderate", "Complex"])

        if pivot.shape[0] < 3:
            print(f"{metric:22s}: not enough paired (KG,Model) rows.")
            continue

        S = pivot["Simple"]
        M = pivot["Moderate"]
        C = pivot["Complex"]

        stat_SM, p_SM = wilcoxon(S, M)
        stat_MC, p_MC = wilcoxon(M, C)
        stat_SC, p_SC = wilcoxon(S, C)

        print(f"{metric}:")
        print(f"  Simple vs Moderate : W = {stat_SM:.3f}, p = {p_SM:.4f}")
        print(f"  Moderate vs Complex: W = {stat_MC:.3f}, p = {p_MC:.4f}")
        print(f"  Simple vs Complex  : W = {stat_SC:.3f}, p = {p_SC:.4f}\n")

In [18]:
def kg_complexity_interaction_tests(h1_c_clean: pd.DataFrame):
    """
    Extra H1 analyses:
      1) Big vs Small KG at each Complexity (Simple/Moderate/Complex)
         - Wilcoxon with Model as the paired unit.
      2) Complexity effect within each KG
         - Friedman across Simple/Moderate/Complex (Model-level).
      3) KG × Complexity interaction proxies:
         - Compare Simple→Moderate drop on Small vs Big
         - Compare Moderate→Complex drop on Small vs Big
         - Compare Simple→Complex drop on Small vs Big

    Assumes a balanced design over the 4 models:
      Models: Claude, GPT5, Deepseek R1, GPT OSS
      KGs:    Small, Big
      Complexity: Simple, Moderate, Complex
    """
    print("\n=== EXTRA: KG × Complexity analyses for H1 metrics ===\n")

    metrics = ["syntax_ok_rate", "satisfiable_rate", "deterministic_rate", "h1_overall"]
    complexities = ["Simple", "Moderate", "Complex"]
    kgs = ["Small", "Big"]

    for metric in metrics:
        print(f"\n--- Metric: {metric} ---")

        # Pivot: index = Model, columns = (KG, Complexity)
        pivot = h1_c_clean.pivot_table(
            index="Model",
            columns=["KG", "Complexity"],
            values=metric
        )

        # Ensure we have all KG/Complexity combos
        missing_cols = []
        for kg in kgs:
            for comp in complexities:
                if (kg, comp) not in pivot.columns:
                    missing_cols.append((kg, comp))
        if missing_cols:
            print(f"  Skipping {metric}: missing cells {missing_cols}")
            continue

        # ------------------------------------------------------
        # (1) Big vs Small at each Complexity level
        # ------------------------------------------------------
        print("  Big vs Small KG at each Complexity (Wilcoxon, Model-level):")
        for comp in complexities:
            small_vals = pivot[("Small", comp)].dropna()
            big_vals   = pivot[("Big", comp)].dropna()

            # Need at least 2 paired models
            common_index = small_vals.index.intersection(big_vals.index)
            if len(common_index) < 2:
                print(f"    {comp:9s}: not enough paired models.")
                continue

            s = small_vals.loc[common_index]
            b = big_vals.loc[common_index]

            stat, p = wilcoxon(s, b)
            print(
                f"    {comp:9s}: "
                f"Small_mean = {s.mean():.3f}, Big_mean = {b.mean():.3f}, "
                f"W = {stat:.3f}, p = {p:.4f}"
            )

        # ------------------------------------------------------
        # (2) Complexity effect within each KG (Friedman)
        # ------------------------------------------------------
        print("  Complexity effect within each KG (Friedman, Model-level):")
        for kg in kgs:
            sub = pivot[kg].copy()  # columns: Simple, Moderate, Complex
            sub = sub.dropna(subset=complexities)
            if sub.shape[0] < 2:
                print(f"    {kg:5s}: not enough complete models for Friedman.")
                continue

            S = sub["Simple"].values
            M = sub["Moderate"].values
            C = sub["Complex"].values

            stat, p = friedmanchisquare(S, M, C)
            print(f"    {kg:5s}: Friedman χ² = {stat:.3f}, p = {p:.4f}")

        # ------------------------------------------------------
        # (3) KG × Complexity interaction proxies:
        #     Simple→Moderate, Moderate→Complex, Simple→Complex
        # ------------------------------------------------------
        print("  KG × Complexity (drops comparison, Small vs Big; Wilcoxon):")

        small_simple   = pivot[("Small", "Simple")].dropna()
        small_moderate = pivot[("Small", "Moderate")].dropna()
        small_complex  = pivot[("Small", "Complex")].dropna()
        big_simple     = pivot[("Big", "Simple")].dropna()
        big_moderate   = pivot[("Big", "Moderate")].dropna()
        big_complex    = pivot[("Big", "Complex")].dropna()

        # Align indices across all six cells
        common_index = (
            small_simple.index
            .intersection(small_moderate.index)
            .intersection(small_complex.index)
            .intersection(big_simple.index)
            .intersection(big_moderate.index)
            .intersection(big_complex.index)
        )

        if len(common_index) < 2:
            print("    Not enough models with all six cells (Small/Big × Simple/Moderate/Complex).")
            continue

        ss = small_simple.loc[common_index]
        sm = small_moderate.loc[common_index]
        sc = small_complex.loc[common_index]
        bs = big_simple.loc[common_index]
        bm = big_moderate.loc[common_index]
        bc = big_complex.loc[common_index]

        # Simple -> Moderate drop
        drop_small_SM = ss - sm
        drop_big_SM   = bs - bm
        stat_sm, p_sm = wilcoxon(drop_small_SM, drop_big_SM)
        print(
            f"    Simple→Moderate drop: "
            f"mean_drop_Small = {drop_small_SM.mean():.3f}, "
            f"mean_drop_Big = {drop_big_SM.mean():.3f}, "
            f"W = {stat_sm:.3f}, p = {p_sm:.4f}"
        )

        # Moderate -> Complex drop
        drop_small_MC = sm - sc
        drop_big_MC   = bm - bc
        stat_mc, p_mc = wilcoxon(drop_small_MC, drop_big_MC)
        print(
            f"    Moderate→Complex drop: "
            f"mean_drop_Small = {drop_small_MC.mean():.3f}, "
            f"mean_drop_Big = {drop_big_MC.mean():.3f}, "
            f"W = {stat_mc:.3f}, p = {p_mc:.4f}"
        )

        # Simple -> Complex drop
        drop_small_SC = ss - sc
        drop_big_SC   = bs - bc
        stat_sc2, p_sc2 = wilcoxon(drop_small_SC, drop_big_SC)
        print(
            f"    Simple→Complex drop: "
            f"mean_drop_Small = {drop_small_SC.mean():.3f}, "
            f"mean_drop_Big = {drop_big_SC.mean():.3f}, "
            f"W = {stat_sc2:.3f}, p = {p_sc2:.4f}"
        )

    print()


In [ ]:
# ========= CONFIG =========
# EXCEL_PATH = Path("all_models_consolidated_results.xlsx")  # change if needed
EXCEL_PATH = Path("balanced_models_consolidated_results.xlsx")  # change if needed

# Total # of CQs and per-complexity split (given by you)
TOTAL_CQS = 33
CQS_PER_COMPLEXITY = {"Simple": 12, "Moderate": 11, "Complex": 10}
OUT_DIR = Path("analysis_plots")  # folder to store PNGs
OUT_DIR.mkdir(exist_ok=True)
# ==========================

print(f"Loading data from: {EXCEL_PATH}")
screen, screen_c, h1, h1_c = load_and_clean(EXCEL_PATH)

screen_c_clean = prepare_screen_with_complexity(screen_c)
h1_c_clean = prepare_h1_with_complexity(h1_c)

# Analyses
sc_df = analyze_screen(screen_c_clean)
per_pair_df = analyze_h1(h1_c_clean)

# Extra analyses (do not change existing ones)
model_sensitivity = summarize_model_sensitivity(per_pair_df)
effect_size_simple_vs_complex(h1_c_clean)
screen_h1_link = correlate_screen_and_h1(sc_df, per_pair_df)
rankings = rank_models_by_complexity(h1_c_clean)

# Plots
print(f"Saving plots to: {OUT_DIR.resolve()}")
plot_screen_boxplots(screen_c_clean)
plot_screen_boxplots_by_kg(screen_c_clean)
plot_h1_boxplots(h1_c_clean)
plot_h1_trendlines(h1_c_clean)
plot_h1_rho_heatmap(per_pair_df)
plot_h1_boxplots_by_kg(h1_c_clean)
plot_h1_boxplots_kg_effect_per_complexity(h1_c_clean)
kg_effect_table = h1_kg_effect_stats(h1_c_clean)
print("Done generating plots.")

# =====================================================================
#  EXTRA TESTS TO STRENGTHEN H1 EVIDENCE
# =====================================================================
friedman_test_h1(h1_c_clean)
jt_test_h1(h1_c_clean)
kendalls_w_h1(h1_c_clean)
pairwise_wilcoxon_h1(h1_c_clean)
kg_complexity_interaction_tests(h1_c_clean)

Loading data from: consolidated_results_filter_only4.xlsx

=== Part A: SCREEN metrics (task adherence) ===

Global Kruskal–Wallis on FV_rate across Complexity:
  H = 2.550, p = 0.2794

Global Kruskal–Wallis on VS_rate (syntax recognition) across Complexity:
  H = 3.016, p = 0.2214

Global Spearman between ComplexityLevel and FV_rate:
  rho = -0.295, p = 0.1610

Global Spearman between ComplexityLevel and VS_rate:
  rho = -0.362, p = 0.0825

Per (KG, Model) Spearman correlations for SCREEN (FV_rate & VS_rate):
   KG       Model  rho_FV     p_FV    rho_VS     p_VS
  Big      Claude    -1.0 0.000000 -1.000000 0.000000
  Big Deepseek R1     0.5 0.666667  0.500000 0.666667
  Big     GPT OSS     1.0 0.000000  0.500000 0.666667
  Big        GPT5    -0.5 0.666667 -1.000000 0.000000
Small      Claude    -1.0 0.000000 -1.000000 0.000000
Small Deepseek R1    -1.0 0.000000 -1.000000 0.000000
Small     GPT OSS    -1.0 0.000000 -0.866025 0.333333
Small        GPT5    -1.0 0.000000 -1.000000 0.000000

/root/.local/lib/python3.10/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/root/.local/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/root/.local/lib/python3.10/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/root/.local/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")
/root/.local/lib/python3.10/site-packages/scipy/stats/_morestats.py:4088: UserWarning: E